In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from DBSCANSupport import DBSCANSupport
from LocalPath import LOCALPATH
from sklearn.neighbors import NearestNeighbors

Load gravity data

In [ ]:
grav = pd.read_csv(LOCALPATH + 'data/test_grav.csv').drop(columns=["old_ind"])
grav = grav[['Latitude', 'Longitude', 'Intensity']]

In [ ]:
lat = list(grav['Latitude'])
longg = list(grav['Longitude'])
inten = list(grav['Intensity'])
#fig.show()

In [ ]:
Z = DBSCANSupport.formatData(grav, 'Intensity')
data = Z

In [ ]:
neighbors = NearestNeighbors(n_neighbors=2)
neighbors_fit = neighbors.fit(grav.to_numpy())
distances, indices = neighbors_fit.kneighbors(data)
distances = np.sort(distances, axis=0)
distances = distances[:,1]
px.line(x=range(0, len(distances)), y=distances).show()

In [ ]:
test_eps = np.linspace(1840, 1850, 20)
test_samp = np.arange(2, 10)
DBModel_test = DBSCANSupport(LOCALPATH+"data/sample_mask.txt.xlsx", train_zone=(-6, -1.5, -98, -90))
validator = DBModel_test.seamounts

In [ ]:
score, params, data_out  = DBModel_test.gridSearch(test_eps, test_samp, data, test='auto', verbose=True)

In [ ]:
dfout = pd.DataFrame(data_out, columns=["Easting", "Northing", "Label", "Intensity"])
DBModel_test.matchPoints(dfout)
df_labeled = dfout[dfout['Label'] == -1]

In [ ]:
fig = px.scatter(dfout, x="Easting", y="Northing", color="Label")
fig.show()

In [ ]:
test_zone=(-6, -1.5, -98, -90)
seamounts = DBModel_test.seamounts
fig3 = px.scatter(x=seamounts[:, 0], y=seamounts[:, 1])
fig3.show()

In [ ]:
fig = px.scatter(df_labeled, x="Easting", y="Northing", color="True_Seamount")
fig.add_trace(px.scatter(x=seamounts[:, 0], y=seamounts[:, 1]).data[0])
fig.update_xaxes(
    scaleanchor="y",
    scaleratio=1,
  )
fig.show()

In [ ]:
fig2 = px.scatter(dfout, x="Easting", y="Northing", color="True_Seamount")
fig2.update_xaxes(
    scaleanchor="y",
    scaleratio=1,
  )

In [ ]:
params